In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


dataset = np.loadtxt('../heartbeat_60s.txt',dtype='float32')
dataset = dataset.astype(np.float32)
max_value = np.max(dataset)
min_value = np.min(dataset)
scalar = max_value - min_value
dataset = list(map(lambda x: x / scalar, dataset))
dataset = dataset[0:1600]
n = 2
def create_dataset(dataset, look_back=n):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

# 创建好输入输出
data_X, data_Y = create_dataset(dataset)

# 划分训练集和测试集，70% 作为训练集
train_size = int(len(data_X) * 0.7)
test_size = len(data_X) - train_size
train_X = data_X[:train_size]
train_Y = data_Y[:train_size]
test_X = data_X[train_size:]
test_Y = data_Y[train_size:]

import torch
train_X = train_X.reshape(-1, 5, n)
train_Y = train_Y.reshape(-1, 5, n)
test_X = test_X.reshape(-1, 1, n)

train_x = torch.from_numpy(train_X)
train_y = torch.from_numpy(train_Y)

from torch import nn
from torch.autograd import Variable

class lstm(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,num_layer):
        super(lstm,self).__init__()
        self.layer1 = nn.LSTM(input_size,hidden_size,num_layer)
        self.dropout = nn.Dropout(0.5)
        self.layer2 = nn.Linear(hidden_size,output_size)
    
    def forward(self,x):
        x,_ = self.layer1(x)
        x = self.dropout(x)
        s,b,h = x.size()
        x = x.view(s*b,h)
        x = self.layer2(x)
        x = x.view(s,b,-1)
        return x

model = lstm(5,4,1,3)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
loss_final = list()
# 开始训练
for e in range(10000):
    var_x = Variable(train_x)
    var_y = Variable(train_y)
    # 前向传播
    out = model(var_x)
    loss = criterion(out, var_y)
    pred = out.argmax(dim=1)

    num_correct = 1
    num_correct = torch.eq(pred, var_y).sum()
    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_final.append(loss.item())
    if (e + 1) % 1 == 0: # 每 100 次输出结果
        print('Epoch: {}, Loss: {:.5f}, Acc: {:.4f}'.format(e + 1, loss.item(),num_correct.item()))

torch.save(model, './heartrate.pkl')
    
model = model.eval() # 转换成测试模式

data_X = data_X.reshape(-1, 1, n)
data_X = torch.from_numpy(data_X)
var_data = Variable(data_X)
pred_test = model(var_data) # 测试集的预测结果
# 改变输出的格式
pred_test = pred_test.view(-1).data.numpy()
# 画出实际结果和预测的结果
plt.plot(pred_test, 'g', label='prediction')
plt.plot(dataset, 'b', label='real')
plt.legend(loc='best')
plt.title('The graph of prediction and real dataset')
plt.show()
plt.savefig('./prediction.png')


ValueError: cannot reshape array of size 2236 into shape (5,2)

In [8]:
var_data

tensor([[[0.8227, 0.8156]],

        [[0.8156, 0.8156]],

        [[0.8156, 0.8298]],

        ...,

        [[0.7376, 0.7447]],

        [[0.7447, 0.7234]],

        [[0.7234, 0.7518]]])